In [1]:
import os
import cv2
import glob
import torch
import shutil
import itertools
import torch.nn as nn
import torch.optim as optim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from pathlib import Path
from torch.nn import functional as F
from torchvision import datasets, models, transforms
# from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
# from pytorch_grad_cam.utils.image import show_cam_on_image

In [2]:
# #FOR NORMAL IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_negative\\'

In [3]:
#FOR NORMAL W/AUG IMAGES
positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\1200aug_equalized_sampled_positive\\'
negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\1200aug_equalized_sampled_negative\\'

In [4]:
# #FOR MASKED IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\negative\\'

In [5]:
os.path.exists(negative_path)

True

In [6]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data\\working\\train')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/train/positive')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test/positive')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/train/negative')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test/negative')

In [7]:
positive_train_len = int(np.floor(len(os.listdir(positive_path))*0.8))
positive_len = len(os.listdir(positive_path))

negative_train_len = int(np.floor(len(os.listdir(negative_path))*0.8))
negative_len = len(os.listdir(negative_path))

In [8]:
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive')
    
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/positive'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len, positive_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/positive')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/negative'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len, negative_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/negative')

In [9]:
normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        normalizer
    ]),
    
    'validation': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        normalizer
    ]),

    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        normalizer
    ])
}

In [10]:
data_images = {
    'train': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/train', data_transforms['train']),
    'validation': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/val', data_transforms['validation']),
    'test': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/test', data_transforms['test'])
}

In [11]:
dataloaders = {
    'train': torch.utils.data.DataLoader(data_images['train'], batch_size=32, shuffle=True, num_workers=0),
    'validation': torch.utils.data.DataLoader(data_images['validation'], batch_size=32,shuffle=True,num_workers=0),
    'test': torch.utils.data.DataLoader(data_images['test'], batch_size=32,shuffle=True,num_workers=0)
}

In [12]:
device_id = 0
torch.cuda.device_count()
torch.cuda.get_device_name(device_id)
# print(device_id)

'NVIDIA GeForce GTX 1660 Ti'

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.aux_logits=False
if torch.cuda.is_available():
    model.cuda()

Using cache found in C:\Users\arnav/.cache\torch\hub\pytorch_vision_v0.10.0


In [14]:
# print(model)

In [15]:
for param in model.parameters():
    param.requires_grad = False

In [16]:
model.fc = nn.Sequential(
    nn.Linear(2048, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 3)
).to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [18]:
# def trained_model(model, criterion, optimizer, epochs):
#     best_acc = 0.0
#     best_precision = 0.0
#     best_recall = 0.0
#     best_f1 = 0.0

#     for epoch in range(epochs):
        
#         print('Epoch:', str(epoch+1) + '/' + str(epochs))
#         print('-'*10)
        
#         for phase in ['train', 'validation']:
#             if phase == 'train':
#                 model.train() #this trains the model
#             else:
#                 model.eval() #this evaluates the model

#             running_loss, running_corrects = 0.0, 0 

#             iteration_count=0
#             total_precision=0
#             total_recall=0
#             total_f1=0
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device) #convert inputs to cpu or cuda
#                 labels = labels.to(device) #convert labels to cpu or cuda

                
#                 # inputs = inputs.logits
#                 outputs = model(inputs) #outputs is inputs being fed to the model
#                 print('This is OUTPUT')
#                 print(outputs)
#                 # outputs = outputs.logits
#                 loss = criterion(outputs, labels) #outputs are fed into the model

#                 if phase == 'train':
#                     optimizer.zero_grad() #sets gradients to zero
#                     loss.backward() #computes sum of gradients
#                     optimizer.step() #preforms an optimization step

#                 _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
#                 running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
#                 running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

#                 iteration_count+=1
#                 precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 total_precision+=precision_it
#                 total_recall+=recall_it
#                 total_f1+=f1_it

#             epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
#             epoch_accuracy = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy
#             epoch_precision=total_precision/iteration_count
#             epoch_recall=total_recall/iteration_count
#             epoch_f1=total_f1/iteration_count

#             if phase == 'validation' and epoch_accuracy > best_acc:
#                 best_acc = epoch_accuracy
#                 best_precision = epoch_precision
#                 best_recall = epoch_recall
#                 best_f1 = epoch_f1

#             print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
#             phase, epoch_loss, epoch_accuracy, epoch_precision, epoch_recall, epoch_f1))
#             # print(phase, ' loss:', epoch_loss, 'epoch_accuracy:', epoch_accuracy)


#     print('Best val Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))
#     return model

In [19]:
def trained_model(model, criterion, optimizer, epochs):
    best_acc = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0

    for epoch in range(epochs):
        
        print('Epoch:', str(epoch+1) + '/' + str(epochs))
        print('-'*10)
        
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train() #this trains the model
            else:
                model.eval() #this evaluates the model

            running_loss, running_corrects = 0.0, 0 

            iteration_count=0
            total_precision=0
            total_recall=0
            total_f1=0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) #convert inputs to cpu or cuda
                labels = labels.to(device) #convert labels to cpu or cuda

                
                # inputs = inputs.logits
                outputs = model(inputs) #outputs is inputs being fed to the model
                # print('This is OUTPUT')
                # print(outputs)
                # outputs = outputs.logits
                loss = criterion(outputs, labels) #outputs are fed into the model

                if phase == 'train':
                    optimizer.zero_grad() #sets gradients to zero
                    loss.backward() #computes sum of gradients
                    optimizer.step() #preforms an optimization step

                _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
                running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
                running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

                iteration_count+=1
                precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                total_precision+=precision_it
                total_recall+=recall_it
                total_f1+=f1_it

            epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
            epoch_accuracy = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy
            epoch_precision=total_precision/iteration_count
            epoch_recall=total_recall/iteration_count
            epoch_f1=total_f1/iteration_count

            if phase == 'test' and epoch_accuracy > best_acc:
                best_acc = epoch_accuracy
                best_precision = epoch_precision
                best_recall = epoch_recall
                best_f1 = epoch_f1

            print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
            phase, epoch_loss, epoch_accuracy, epoch_precision, epoch_recall, epoch_f1))
            # print(phase, ' loss:', epoch_loss, 'epoch_accuracy:', epoch_accuracy)


    print('Best test Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))
    return model

In [20]:
model = trained_model(model, criterion, optimizer, 30)

Epoch: 1/30
----------


C:\Users\arnav\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.396 Acc: 0.823 Prec: 0.850 Recall: 0.823 F1 0.817
test Loss: 0.198 Acc: 0.928 Prec: 0.940 Recall: 0.931 F1 0.931
Epoch: 2/30
----------
train Loss: 0.294 Acc: 0.878 Prec: 0.896 Recall: 0.878 F1 0.878
test Loss: 0.180 Acc: 0.936 Prec: 0.949 Recall: 0.938 F1 0.938
Epoch: 3/30
----------
train Loss: 0.274 Acc: 0.884 Prec: 0.896 Recall: 0.884 F1 0.884
test Loss: 0.167 Acc: 0.941 Prec: 0.953 Recall: 0.943 F1 0.943
Epoch: 4/30
----------
train Loss: 0.267 Acc: 0.888 Prec: 0.901 Recall: 0.888 F1 0.888
test Loss: 0.128 Acc: 0.961 Prec: 0.964 Recall: 0.960 F1 0.960
Epoch: 5/30
----------
train Loss: 0.268 Acc: 0.888 Prec: 0.905 Recall: 0.888 F1 0.888
test Loss: 0.163 Acc: 0.934 Prec: 0.944 Recall: 0.936 F1 0.936
Epoch: 6/30
----------
train Loss: 0.270 Acc: 0.889 Prec: 0.904 Recall: 0.889 F1 0.889
test Loss: 0.164 Acc: 0.943 Prec: 0.953 Recall: 0.944 F1 0.945
Epoch: 7/30
----------
train Loss: 0.266 Acc: 0.891 Prec: 0.905 Recall: 0.891 F1 0.891
test Loss: 0.171 Acc: 0.939 Prec: 0.

In [21]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/models')

In [22]:
torch.save(model.state_dict(), 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_weights/working/models/weights.h5') #save the model's weights
# test_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)

# test_model.fc = nn.Sequential(
#     nn.Linear(2048, 64),
#     nn.ReLU(inplace=True),
#     nn.Linear(64, 3)
# ).to(torch.device("cuda:0"))

model.load_state_dict(torch.load('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_weights/working/models/weights.h5')) #load the model's weights
# test_model.cuda()

<All keys matched successfully>

In [23]:
# def test_model_func(model):
    
#     num_correct = 0
#     num_samples = 0
#     model.eval()


#     # best_acc = 0.0
#     # best_precision = 0.0
#     # best_recall = 0.0
#     # best_f1 = 0.0

#     # running_loss, running_corrects = 0.0, 0 
#     # iteration_count=0
#     # total_precision=0
#     # total_recall=0
#     # total_f1=0
#     with torch.no_grad():
#         for inputs, labels in dataloaders['test']:



#             inputs = inputs.to(torch.device("cuda:0")) #convert inputs to cpu or cuda
#             labels = labels.to(torch.device("cuda:0")) #convert labels to cpu or cuda

#             # inputs = inputs.logits
#             outputs = model(inputs) #outputs is inputs being fed to the model
#             # outputs = outputs.logits
#             # loss = criterion(outputs, labels) #outputs are fed into the model

#             # scores = model(inputs)
#             # _, predictions = scores.max(1)
#             _, predictions = torch.max(outputs, 1)
#             num_correct += (predictions == labels).sum()
#             num_samples += predictions.size(0)

#         print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
#     model.train()
#             # _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
#             # running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
#             # running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

#             # iteration_count+=1
#             # precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#             # recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#             # f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#             # total_precision+=precision_it
#             # total_recall+=recall_it
#             # total_f1+=f1_it

#     # epoch_loss = running_loss / len(data_images['test']) #this is the epoch loss
#     # epoch_accuracy = running_corrects.double() / len(data_images['test']) #this is the epoch accuracy
#     # epoch_precision=total_precision/iteration_count
#     # epoch_recall=total_recall/iteration_count
#     # epoch_f1=total_f1/iteration_count

#     # if epoch_accuracy > best_acc:
#     #     best_acc = epoch_accuracy
#     #     best_precision = epoch_precision
#     #     best_recall = epoch_recall
#     #     best_f1 = epoch_f1

#     # print('Test Loss: {:.3f}, Test Acc: {:.3f}, Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}'.format(epoch_loss, best_acc, best_precision, best_recall, best_f1))





In [24]:
# def test_model_func(model):

#     model.eval()

#     phase = 'test'
    
#     running_loss, running_corrects = 0.0, 0 

#     iteration_count=0
#     total_precision=0
#     total_recall=0
#     total_f1=0
#     for inputs, labels in dataloaders[phase]:
#         inputs = inputs.to('cpu') #convert inputs to cpu or cuda
#         labels = labels.to('cpu') #convert labels to cpu or cuda
#         # print('This is INPUT')
#         # print(inputs)
#         # print('This is LABELS')
#         # print(labels)


        
#         # inputs = inputs.logits
#         outputs = model(inputs) #outputs is inputs being fed to the model
#         # print('This is OUTPUT')
#         # print(outputs)
#         # outputs = outputs.logits
#         loss = criterion(outputs, labels) #outputs are fed into the model

#         _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
#         running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
#         running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

#         iteration_count+=1
#         precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#         recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#         f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#         total_precision+=precision_it
#         total_recall+=recall_it
#         total_f1+=f1_it


#         epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
#         epoch_accuracy = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy
#         epoch_precision=total_precision/iteration_count
#         epoch_recall=total_recall/iteration_count
#         epoch_f1=total_f1/iteration_count

#         if phase == 'validation' and epoch_accuracy > best_acc:
#             best_acc = epoch_accuracy
#             best_precision = epoch_precision
#             best_recall = epoch_recall
#             best_f1 = epoch_f1

#         print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
#         phase, epoch_loss, epoch_accuracy, epoch_precision, epoch_recall, epoch_f1))
#         # print(phase, ' loss:', epoch_loss, 'epoch_accuracy:', epoch_accuracy)


#     print('Best val Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))

In [25]:
# from GPUtil import showUtilization as gpu_usage
# gpu_usage()

In [26]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [27]:
# print(device)

In [28]:
# test_model_func(test_model)